In [3]:
import re
import pandas as pd
import numpy as np
import sqlalchemy
import xlsxwriter

from sqlalchemy import create_engine
from pathlib import Path
from datetime import datetime, timedelta

In [4]:
dias_semana = { 'Sunday'    : 'Domingo', 
                'Monday'    : 'Segunda', 
                'Tuesday'   :'Terça', 
                'Wednesday' : 'Quarta', 
                'Thursday'  : 'Quinta', 
                'Friday' : 'Sexta', 
                'Saturday' :'Sabádo'}

In [5]:
month = '1'
year  = '2019'

In [6]:
engine = create_engine("sqlite:///registros.db", echo=False)

In [7]:
pis_codes = engine.execute(f"SELECT DISTINCT pis FROM registros WHERE year = {year} AND month = {month}").fetchall()

print(len(pis_codes))

pis_codes

10


[('012303844632',),
 ('019004076622',),
 ('017048753097',),
 ('016002684248',),
 ('013979783277',),
 ('020637703280',),
 ('021065315459',),
 ('020071346877',),
 ('012699519512',),
 ('013118171501',)]

In [8]:
pis_code = pis_codes[3][0]

pis_code

'016002684248'

In [9]:
engine.execute(f"SELECT DISTINCT name FROM funcionarios WHERE pis = '{pis_code}'").fetchall()

[('DANIELE DE LIMA ALVES',)]

In [10]:
query = f"""SELECT * 
            FROM registros 
            WHERE year = {year} AND month = {month} AND pis = '{pis_code}'
        """

df = pd.read_sql(query, 
                 engine,
                 parse_dates='datetime')

df['date'] = df['datetime'].dt.date
df['rank'] = df.groupby(df.datetime.dt.day)['datetime'].rank().astype(int)

df.head(10)

,sequential_number,cod_op,day,month,year,hour,minutes,cod_input,pis,name,datetime,date,rank
0,35079,3,7,1,2019,8,5,None,016002684248,None,2019-01-07 08:05:00,2019-01-07,1
1,35081,3,7,1,2019,12,7,None,016002684248,None,2019-01-07 12:07:00,2019-01-07,2
2,35088,3,7,1,2019,13,5,None,016002684248,None,2019-01-07 13:05:00,2019-01-07,3
3,35091,3,7,1,2019,17,0,None,016002684248,None,2019-01-07 17:00:00,2019-01-07,4
4,35100,3,8,1,2019,8,8,None,016002684248,None,2019-01-08 08:08:00,2019-01-08,1
5,35104,3,8,1,2019,12,18,None,016002684248,None,2019-01-08 12:18:00,2019-01-08,2
6,35109,3,8,1,2019,17,0,None,016002684248,None,2019-01-08 17:00:00,2019-01-08,3
7,35116,3,9,1,2019,8,16,None,016002684248,None,2019-01-09 08:16:00,2019-01-09,1
8,35122,3,9,1,2019,12,45,None,016002684248,None,2019-01-09 12:45:00,2019-01-09,2
9,35124,3,9,1,2019,13,2,None,016002684248,None,2019-01-09 13:02:00,2019-01-09,3


In [11]:
table = df.pivot(index=['date'], columns='rank', values='datetime')

columns_reg = list(table.columns)

print(columns_reg)

# table

[1, 2, 3, 4]


In [12]:
tmp = df.groupby('date', as_index=True)['rank'].max()

print(tmp.name)

s1 = (tmp > 0) & (tmp <= 6) & (tmp % 2 == 0)
s1.name = 'is_valid'

rank


In [13]:
index = pd.date_range(start='2019-05-01', end='2019-05-31')

# index.day_name()

df = pd.DataFrame(index.day_name(), index=index, columns=['WeekName'])

# df

In [14]:
table2 = df.merge(table, right_index=True, left_index=True, how='left')

In [15]:
table3 = table2.merge(s1, right_index=True, left_index=True, how='left')

table3['is_valid'] = table3['is_valid'].fillna(False)

table3['WeekName'] = table3['WeekName'].map(dias_semana)

In [16]:
table3

,WeekName,1,2,3,4,is_valid
2019-05-01,Quarta,NaT,NaT,NaT,NaT,False
2019-05-02,Quinta,NaT,NaT,NaT,NaT,False
2019-05-03,Sexta,NaT,NaT,NaT,NaT,False
2019-05-04,Sabádo,NaT,NaT,NaT,NaT,False
2019-05-05,Domingo,NaT,NaT,NaT,NaT,False
2019-05-06,Segunda,NaT,NaT,NaT,NaT,False
2019-05-07,Terça,NaT,NaT,NaT,NaT,False
2019-05-08,Quarta,NaT,NaT,NaT,NaT,False
2019-05-09,Quinta,NaT,NaT,NaT,NaT,False
2019-05-10,Sexta,NaT,NaT,NaT,NaT,False


In [17]:
writer = pd.ExcelWriter("exemplo2.xlsx",
                        datetime_format='hh:mm',
                        engine='xlsxwriter',)

planilhas = ['original', 'validada']

for plan in planilhas:
    # copy the same data in two sheets

    (table3.reset_index().to_excel(writer, sheet_name=plan, startrow=2, index=False))

    wb = writer.book
    ws = writer.sheets[plan]

    ws.write('A1', 'Servidor:')
    ws.write('B1', 'Maria de Jesus')

    # fmt1 = wb.add_format({'num_format' : 'dd/mm/yyyy'})


    ws.set_column('A:A', 20)
    ws.set_column('B:B', 15)
    ws.set_column('C:H', 10)
    ws.set_column('I:I', 15)
    ws.set_column('J:J', 5)
    ws.set_column('N:N', 5)
    start_cell    = 4
    max_row = len(table3)
    fmt2 = wb.add_format({'num_format': 'hh:mm:ss'}) # 

    ws.write(f'K{start_cell - 1}', 'Manhã')
    ws.write(f'L{start_cell - 1}', 'Tarde')
    ws.write(f'M{start_cell - 1}', 'Noite')
    ws.write(f'O{start_cell - 1}', 'Total Dia')

    for i in range(0, max_row):
        row = i + start_cell

        cell = f'K{row}'
        formula  = f'=D{row} - C{row}'
        ws.write_formula(cell, formula, fmt2)

        cell = f'L{row}'
        formula  = f'=F{row} - E{row}'
        ws.write_formula(cell, formula, fmt2)

        cell = f'M{row}'
        formula  = f'=H{row} - G{row}'
        ws.write_formula(cell, formula, fmt2)

        cell = f'O{row}'
        formula = f'K{row} + L{row} + M{row}'
        ws.write_formula(cell, formula, fmt2)

writer.save()

In [28]:
len(table3)

31

In [17]:
try:
    pd.date_range(start='2019-04-01', end='2019-04-31')
except ValueError as error :
    print(str(error) == "could not convert string to Timestamp")

True
